In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [2]:
def monte_carlo_option_pricing(S0, K, T, r, sigma, num_simulations):
    """
    Monte Carlo simulation for European call option pricing.

    Parameters:
    S0 : float
        Initial stock price
    K : float
        Strike price
    T : float
        Time to maturity (in years)
    r : float
        Risk-free interest rate
    sigma : float
        Volatility of the underlying asset
    num_simulations : int
        Number of simulations

    Returns:
    float
        Estimated option price
    """
    # Generate random standard normal variables
    Z = np.random.standard_normal(num_simulations)
    # Calculate stock price at maturity
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    # Calculate the payoff at maturity
    payoffs = np.maximum(ST - K, 0)
    # Discount back to present value
    option_price = np.exp(-r * T) * np.mean(payoffs)
    return option_price

In [3]:
def black_scholes_option_price(S0, K, T, r, sigma):
    """
    Black-Scholes formula for European call option pricing.

    Parameters:
    S0 : float
        Initial stock price
    K : float
        Strike price
    T : float
        Time to maturity (in years)
    r : float
        Risk-free interest rate
    sigma : float
        Volatility of the underlying asset

    Returns:
    float
        Option price
    """
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_price = (S0 * norm.cdf(d1)) - (K * np.exp(-r * T) * norm.cdf(d2))
    return call_price

In [7]:
# Parameters
S0 = 100          # Initial stock price
K = 100           # Strike price
T = 1.0           # Time to maturity (1 year)
r = 0.05          # Risk-free interest rate
sigma = 0.3       # Volatility
num_simulations = 100000  # Number of Monte Carlo simulations

In [8]:
# Calculate option prices
mc_price = monte_carlo_option_pricing(S0, K, T, r, sigma, num_simulations)
bs_price = black_scholes_option_price(S0, K, T, r, sigma)

In [9]:
# Store results in a pandas DataFrame
results = pd.DataFrame({
    'Method': ['Monte Carlo', 'Black-Scholes'],
    'Option Price': [mc_price, bs_price]
})

print(results)

          Method  Option Price
0    Monte Carlo     14.245291
1  Black-Scholes     14.231255
